In [1]:
!pip install nltk

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from nltk.tokenize import word_tokenize
import nltk

In [3]:
document = """About the Program
What is the course fee for  Data Science Mentorship Program (DSMP 2023)
The course follows a monthly subscription model where you have to make monthly payments of Rs 799/month.
What is the total duration of the course?
The total duration of the course is 7 months. So the total course fee becomes 799*7 = Rs 5600(approx.)
What is the syllabus of the mentorship program?
We will be covering the following modules:
Python Fundamentals
Python libraries for Data Science
Data Analysis
SQL for Data Science
Maths for Machine Learning
ML Algorithms
Practical ML
MLOPs
Case studies
You can check the detailed syllabus here - https://learnwith.campusx.in/courses/CampusX-Data-Science-Mentorship-Program-637339afe4b0615a1bbed390
Will Deep Learning and NLP be a part of this program?
No, NLP and Deep Learning both are not a part of this program’s curriculum.
What if I miss a live session? Will I get a recording of the session?
Yes all our sessions are recorded, so even if you miss a session you can go back and watch the recording.
Where can I find the class schedule?
Checkout this google sheet to see month by month time table of the course - https://docs.google.com/spreadsheets/d/16OoTax_A6ORAeCg4emgexhqqPv3noQPYKU7RJ6ArOzk/edit?usp=sharing.
What is the time duration of all the live sessions?
Roughly, all the sessions last 2 hours.
What is the language spoken by the instructor during the sessions?
Hinglish
How will I be informed about the upcoming class?
You will get a mail from our side before every paid session once you become a paid user.
Can I do this course if I am from a non-tech background?
Yes, absolutely.
I am late, can I join the program in the middle?
Absolutely, you can join the program anytime.
If I join/pay in the middle, will I be able to see all the past lectures?
Yes, once you make the payment you will be able to see all the past content in your dashboard.
Where do I have to submit the task?
You don’t have to submit the task. We will provide you with the solutions, you have to self evaluate the task yourself.
Will we do case studies in the program?
Yes.
Where can we contact you?
You can mail us at nitish.campusx@gmail.com
Payment/Registration related questions
Where do we have to make our payments? Your YouTube channel or website?
You have to make all your monthly payments on our website. Here is the link for our website - https://learnwith.campusx.in/
Can we pay the entire amount of Rs 5600 all at once?
Unfortunately no, the program follows a monthly subscription model.
What is the validity of monthly subscription? Suppose if I pay on 15th Jan, then do I have to pay again on 1st Feb or 15th Feb
15th Feb. The validity period is 30 days from the day you make the payment. So essentially you can join anytime you don’t have to wait for a month to end.
What if I don’t like the course after making the payment. What is the refund policy?
You get a 7 days refund period from the day you have made the payment.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmail.com
Post registration queries
Till when can I view the paid videos on the website?
This one is tricky, so read carefully. You can watch the videos till your subscription is valid. Suppose you have purchased subscription on 21st Jan, you will be able to watch all the past paid sessions in the period of 21st Jan to 20th Feb. But after 21st Feb you will have to purchase the subscription again.
But once the course is over and you have paid us Rs 5600(or 7 installments of Rs 799) you will be able to watch the paid sessions till Aug 2024.
Why lifetime validity is not provided?
Because of the low course fee.
Where can I reach out in case of a doubt after the session?
You will have to fill a google form provided in your dashboard and our team will contact you for a 1 on 1 doubt clearance session
If I join the program late, can I still ask past week doubts?
Yes, just select past week doubt in the doubt clearance google form.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmai.com
Certificate and Placement Assistance related queries
What is the criteria to get the certificate?
There are 2 criterias:
You have to pay the entire fee of Rs 5600
You have to attempt all the course assessments.
I am joining late. How can I pay payment of the earlier months?
You will get a link to pay fee of earlier months in your dashboard once you pay for the current month.
I have read that Placement assistance is a part of this program. What comes under Placement assistance?
This is to clarify that Placement assistance does not mean Placement guarantee. So we dont guarantee you any jobs or for that matter even interview calls. So if you are planning to join this course just for placements, I am afraid you will be disappointed. Here is what comes under placement assistance
Portfolio Building sessions
Soft skill sessions
Sessions with industry mentors
Discussion on Job hunting strategies
"""


It is unsupervised data we need to convert it to supervised with labels. We can do it like making two one `X` for input and `y` for output:

```
Original: "What is the course fee"
N=3 (context window):
X1: ["What", "is", "the"] → y1: "course"
X2: ["is", "the", "course"] → y1: "fee"
```

Like this our data will have the label.

In [4]:
# Tokenization using punkt tokenizer
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
# Tokenize
tokens = word_tokenize(document.lower())

In [6]:
# Build a vocabulary
vocab = {'<unk>':0}  # Initialize vocabulary with unknown token at index 0

# Iterate through unique tokens from the Counter object
for token in Counter(tokens).keys():
  # Add token to vocabulary if not already present
  if token not in vocab:
    vocab[token] = len(vocab)  # Assign next available index

# Result: vocab dictionary maps each unique word to a unique integer ID

In [8]:
# Extract sentence from data
input_sentences = document.split('\n')

In [9]:
input_sentences

['About the Program',
 'What is the course fee for  Data Science Mentorship Program (DSMP 2023)',
 'The course follows a monthly subscription model where you have to make monthly payments of Rs 799/month.',
 'What is the total duration of the course?',
 'The total duration of the course is 7 months. So the total course fee becomes 799*7 = Rs 5600(approx.)',
 'What is the syllabus of the mentorship program?',
 'We will be covering the following modules:',
 'Python Fundamentals',
 'Python libraries for Data Science',
 'Data Analysis',
 'SQL for Data Science',
 'Maths for Machine Learning',
 'ML Algorithms',
 'Practical ML',
 'MLOPs',
 'Case studies',
 'You can check the detailed syllabus here - https://learnwith.campusx.in/courses/CampusX-Data-Science-Mentorship-Program-637339afe4b0615a1bbed390',
 'Will Deep Learning and NLP be a part of this program?',
 'No, NLP and Deep Learning both are not a part of this program’s curriculum.',
 'What if I miss a live session? Will I get a recording 

In [10]:
def text_to_indices(sentence, vocab):
  """
  Convert a sentence (list of tokens) to numerical indices using vocabulary mapping.

  Parameters:
  sentence (list): List of string tokens to convert
  vocab (dict): Dictionary mapping tokens to integer indices

  Returns:
  list: List of integer indices representing the sentence
  """
  numerical_sentence = []  # Initialize empty list for numerical representation

  for token in sentence:
    # Check if token exists in vocabulary
    if token in vocab:
      # If token exists, append its corresponding index
      numerical_sentence.append(vocab[token])
    else:
      # If token is unknown, append index of '<unk>' (unknown token)
      numerical_sentence.append(vocab['<unk>'])

  return numerical_sentence  # Return the numerical representation

In [11]:
input_numerical_sentences = []

for sentence in input_sentences:
  input_numerical_sentences.append(text_to_indices(word_tokenize(sentence.lower()), vocab))

In [12]:
input_numerical_sentences

[[1, 2, 3],
 [4, 5, 2, 6, 7, 8, 9, 10, 11, 3, 12, 13, 14, 15],
 [2, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 18, 26, 27, 28, 29, 30],
 [4, 5, 2, 31, 32, 27, 2, 6, 33],
 [2,
  31,
  32,
  27,
  2,
  6,
  5,
  34,
  35,
  30,
  36,
  2,
  31,
  6,
  7,
  37,
  38,
  39,
  34,
  40,
  28,
  41,
  12,
  42,
  30,
  15],
 [4, 5, 2, 43, 27, 2, 11, 3, 33],
 [44, 45, 46, 47, 2, 48, 49, 50],
 [51, 52],
 [51, 53, 8, 9, 10],
 [9, 54],
 [55, 8, 9, 10],
 [56, 8, 57, 58],
 [59, 60],
 [61, 59],
 [62],
 [63, 64],
 [22, 65, 66, 2, 67, 43, 68, 69, 70, 50, 71],
 [45, 72, 58, 73, 74, 46, 17, 75, 27, 76, 3, 33],
 [77, 78, 74, 73, 72, 58, 79, 80, 81, 17, 75, 27, 76, 3, 82, 83, 84, 30],
 [4, 85, 86, 87, 17, 88, 89, 33, 45, 86, 90, 17, 91, 27, 2, 89, 33],
 [92,
  93,
  94,
  95,
  80,
  96,
  78,
  36,
  97,
  85,
  22,
  87,
  17,
  89,
  22,
  65,
  98,
  99,
  73,
  100,
  2,
  91,
  30],
 [21, 65, 86, 101, 2, 102, 103, 33],
 [104,
  76,
  105,
  106,
  24,
  107,
  108,
  109,
  108,
  110,
  111,
  27,

In [13]:
len(input_numerical_sentences)

78

Now we will form the sequence like,

For sentence S = `[w₁, w₂, w₃, w₄]`:


```
Iteration 1: [w₁, w₂]      → Predict w₂ given w₁
Iteration 2: [w₁, w₂, w₃]  → Predict w₃ given [w₁, w₂]
Iteration 3: [w₁, w₂, w₃, w₄] → Predict w₄ given [w₁, w₂, w₃]
```

Sentence: `["What", "is", "the", "course", "fee"]`
Created sequences:

`["What", "is"]` (predict "is")

`["What", "is", "the"]` (predict "the")

`["What", "is", "the", "course"]` (predict "course")

`["What", "is", "the", "course", "fee"]` (predict "fee")

In [13]:
for sentence in input_numerical_sentences:

  for i in range(1, len(sentence)):
    print(sentence[:i+1])

[1, 2]
[1, 2, 3]
[4, 5]
[4, 5, 2]
[4, 5, 2, 6]
[4, 5, 2, 6, 7]
[4, 5, 2, 6, 7, 8]
[4, 5, 2, 6, 7, 8, 9]
[4, 5, 2, 6, 7, 8, 9, 10]
[4, 5, 2, 6, 7, 8, 9, 10, 11]
[4, 5, 2, 6, 7, 8, 9, 10, 11, 3]
[4, 5, 2, 6, 7, 8, 9, 10, 11, 3, 12]
[4, 5, 2, 6, 7, 8, 9, 10, 11, 3, 12, 13]
[4, 5, 2, 6, 7, 8, 9, 10, 11, 3, 12, 13, 14]
[4, 5, 2, 6, 7, 8, 9, 10, 11, 3, 12, 13, 14, 15]
[2, 6]
[2, 6, 16]
[2, 6, 16, 17]
[2, 6, 16, 17, 18]
[2, 6, 16, 17, 18, 19]
[2, 6, 16, 17, 18, 19, 20]
[2, 6, 16, 17, 18, 19, 20, 21]
[2, 6, 16, 17, 18, 19, 20, 21, 22]
[2, 6, 16, 17, 18, 19, 20, 21, 22, 23]
[2, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24]
[2, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
[2, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 18]
[2, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 18, 26]
[2, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 18, 26, 27]
[2, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 18, 26, 27, 28]
[2, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 18, 26, 27, 28, 29]
[2, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24, 

In [14]:
# Initialize an empty list to store all training sequences
training_sequence = []

# Iterate through each sentence that has been converted to numerical indices
for sentence in input_numerical_sentences:

  # For each sentence, create progressively longer subsequences
  # Start from i=1 (minimum 2 words) up to full sentence length
  for i in range(1, len(sentence)):
    # Extract subsequence from beginning up to position i (inclusive)
    # Creates growing context windows for next-word prediction
    training_sequence.append(sentence[:i+1])

In [15]:
training_sequence

[[1, 2],
 [1, 2, 3],
 [4, 5],
 [4, 5, 2],
 [4, 5, 2, 6],
 [4, 5, 2, 6, 7],
 [4, 5, 2, 6, 7, 8],
 [4, 5, 2, 6, 7, 8, 9],
 [4, 5, 2, 6, 7, 8, 9, 10],
 [4, 5, 2, 6, 7, 8, 9, 10, 11],
 [4, 5, 2, 6, 7, 8, 9, 10, 11, 3],
 [4, 5, 2, 6, 7, 8, 9, 10, 11, 3, 12],
 [4, 5, 2, 6, 7, 8, 9, 10, 11, 3, 12, 13],
 [4, 5, 2, 6, 7, 8, 9, 10, 11, 3, 12, 13, 14],
 [4, 5, 2, 6, 7, 8, 9, 10, 11, 3, 12, 13, 14, 15],
 [2, 6],
 [2, 6, 16],
 [2, 6, 16, 17],
 [2, 6, 16, 17, 18],
 [2, 6, 16, 17, 18, 19],
 [2, 6, 16, 17, 18, 19, 20],
 [2, 6, 16, 17, 18, 19, 20, 21],
 [2, 6, 16, 17, 18, 19, 20, 21, 22],
 [2, 6, 16, 17, 18, 19, 20, 21, 22, 23],
 [2, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24],
 [2, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25],
 [2, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 18],
 [2, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 18, 26],
 [2, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 18, 26, 27],
 [2, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 18, 26, 27, 28],
 [2, 6, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 18

In [16]:
len(training_sequence)

942

In [17]:
training_sequence[:5]

[[1, 2], [1, 2, 3], [4, 5], [4, 5, 2], [4, 5, 2, 6]]

In [18]:
len_list = []

for sequence in training_sequence:
  len_list.append(len(sequence))

max(len_list)

62

In [19]:
training_sequence[0]

[1, 2]

So maximum length of sequence is 62 so for example in our first sequence there is `[1, 2]` so we need to fill 60 before it with padding of zeros so we can make all sequences even in size.

In [20]:
padded_training_sequence = []
for sequence in training_sequence:

  padded_training_sequence.append([0]*(max(len_list) - len(sequence)) + sequence)

In [21]:
len(padded_training_sequence[10])

62

In [22]:
# Finally convert it to tensor
padded_training_sequence = torch.tensor(padded_training_sequence, dtype=torch.long)

In [23]:
padded_training_sequence

tensor([[  0,   0,   0,  ...,   0,   1,   2],
        [  0,   0,   0,  ...,   1,   2,   3],
        [  0,   0,   0,  ...,   0,   4,   5],
        ...,
        [  0,   0,   0,  ..., 285, 176, 286],
        [  0,   0,   0,  ..., 176, 286, 287],
        [  0,   0,   0,  ..., 286, 287, 288]])

In [24]:
# Split sequences into input (X) and target (y) for next-word prediction
# X contains all words except last (context), y contains only last word (target)
X = padded_training_sequence[:, :-1]  # All columns except last → Input context
y = padded_training_sequence[:, -1]   # Only last column → Word to predict

# Example: Sequence [1,2,3,4] → X=[1,2,3], y=[4]
# Model learns: given X (context), predict y (next word)

In [25]:
X

tensor([[  0,   0,   0,  ...,   0,   0,   1],
        [  0,   0,   0,  ...,   0,   1,   2],
        [  0,   0,   0,  ...,   0,   0,   4],
        ...,
        [  0,   0,   0,  ...,   0, 285, 176],
        [  0,   0,   0,  ..., 285, 176, 286],
        [  0,   0,   0,  ..., 176, 286, 287]])

In [26]:
y

tensor([  2,   3,   5,   2,   6,   7,   8,   9,  10,  11,   3,  12,  13,  14,
         15,   6,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  18,  26,
         27,  28,  29,  30,   5,   2,  31,  32,  27,   2,   6,  33,  31,  32,
         27,   2,   6,   5,  34,  35,  30,  36,   2,  31,   6,   7,  37,  38,
         39,  34,  40,  28,  41,  12,  42,  30,  15,   5,   2,  43,  27,   2,
         11,   3,  33,  45,  46,  47,   2,  48,  49,  50,  52,  53,   8,   9,
         10,  54,   8,   9,  10,   8,  57,  58,  60,  59,  64,  65,  66,   2,
         67,  43,  68,  69,  70,  50,  71,  72,  58,  73,  74,  46,  17,  75,
         27,  76,   3,  33,  78,  74,  73,  72,  58,  79,  80,  81,  17,  75,
         27,  76,   3,  82,  83,  84,  30,  85,  86,  87,  17,  88,  89,  33,
         45,  86,  90,  17,  91,  27,   2,  89,  33,  93,  94,  95,  80,  96,
         78,  36,  97,  85,  22,  87,  17,  89,  22,  65,  98,  99,  73, 100,
          2,  91,  30,  65,  86, 101,   2, 102, 103,  33,  76, 1

In [27]:
class CustomDataset(Dataset):

  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]

In [28]:
dataset = CustomDataset(X,y)

In [29]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

```
Input Sentence (indices)
    ↓
[What, is, the] → [1, 2, 3]
    ↓
Embedding Layer (100D vectors)
    ↓
[[0.1, 0.2, ...], [0.3, 0.4, ...], [0.5, 0.6, ...]]
    ↓
LSTM Layer (processes sequentially)
    ↓
Hidden States: [h₁, h₂, h₃] → Keep last h₃
    ↓
Linear Layer (150 → vocab_size)
    ↓
Output: [p₁, p₂, ..., p_vocab] (probabilities for next word)
```

In [30]:
class LSTMModel(nn.Module):
    """
    LSTM Neural Network for Next-Word Prediction

    Architecture:
    Input → Embedding → LSTM → Linear → Output
    """

    def __init__(self, vocab_size):
        """Initialize model layers"""
        super().__init__()  # Initialize parent nn.Module class

        # Word Embedding Layer: Converts word indices to dense vectors
        # Input: vocab_size (number of unique words)
        # Output: 100-dimensional vectors
        self.embedding = nn.Embedding(vocab_size, 100)

        # LSTM Layer: Processes sequential data
        # Input: 100-dim vectors from embedding
        # Output: 150-dim hidden states
        # batch_first=True: Input shape is (batch, seq_len, features)
        self.lstm = nn.LSTM(100, 150, batch_first=True)

        # Fully Connected Layer: Maps LSTM output to vocabulary size
        # Input: 150-dim hidden state
        # Output: vocab_size (probability distribution over vocabulary)
        self.fc = nn.Linear(150, vocab_size)

    def forward(self, x):
        """Forward pass through the network"""

        # Step 1: Convert word indices to embeddings
        # x shape: (batch_size, sequence_length)
        # embedded shape: (batch_size, sequence_length, embedding_dim=100)
        embedded = self.embedding(x)

        # Step 2: Process through LSTM
        # Returns:
        #   intermediate_hidden_states: All hidden states for each time step
        #   (final_hidden_state, final_cell_state): Last time step states
        intermediate_hidden_states, (final_hidden_state, final_cell_state) = self.lstm(embedded)

        # Step 3: Final prediction using last hidden state
        # final_hidden_state shape: (1, batch_size, hidden_size=150)
        # .squeeze(0) removes the first dimension: (batch_size, 150)
        # fc layer outputs: (batch_size, vocab_size)
        output = self.fc(final_hidden_state.squeeze(0))

        return output

In [31]:
model = LSTMModel(len(vocab))

In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [33]:
model.to(device)

LSTMModel(
  (embedding): Embedding(289, 100)
  (lstm): LSTM(100, 150, batch_first=True)
  (fc): Linear(in_features=150, out_features=289, bias=True)
)

In [34]:
epochs = 50
learning_rate = 0.001

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [35]:
# Training loop

for epoch in range(epochs):
  total_loss = 0

  for batch_x, batch_y in dataloader:

    batch_x, batch_y = batch_x.to(device), batch_y.to(device)

    optimizer.zero_grad()

    output = model(batch_x)

    loss = criterion(output, batch_y)

    loss.backward()

    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch + 1}, Loss: {total_loss:.4f}")

Epoch: 1, Loss: 165.5876
Epoch: 2, Loss: 147.4462
Epoch: 3, Loss: 135.2602
Epoch: 4, Loss: 123.6803
Epoch: 5, Loss: 112.5492
Epoch: 6, Loss: 101.8827
Epoch: 7, Loss: 91.8123
Epoch: 8, Loss: 82.3395
Epoch: 9, Loss: 73.8928
Epoch: 10, Loss: 65.6773
Epoch: 11, Loss: 58.6359
Epoch: 12, Loss: 51.8919
Epoch: 13, Loss: 46.1930
Epoch: 14, Loss: 40.8151
Epoch: 15, Loss: 36.0520
Epoch: 16, Loss: 31.7951
Epoch: 17, Loss: 27.9488
Epoch: 18, Loss: 24.9663
Epoch: 19, Loss: 22.4189
Epoch: 20, Loss: 19.6491
Epoch: 21, Loss: 17.6208
Epoch: 22, Loss: 15.9589
Epoch: 23, Loss: 14.5837
Epoch: 24, Loss: 13.2994
Epoch: 25, Loss: 12.1526
Epoch: 26, Loss: 11.1091
Epoch: 27, Loss: 10.3945
Epoch: 28, Loss: 9.8567
Epoch: 29, Loss: 9.0644
Epoch: 30, Loss: 8.5196
Epoch: 31, Loss: 8.0825
Epoch: 32, Loss: 7.8206
Epoch: 33, Loss: 7.3253
Epoch: 34, Loss: 6.9424
Epoch: 35, Loss: 6.7936
Epoch: 36, Loss: 6.4623
Epoch: 37, Loss: 6.2384
Epoch: 38, Loss: 6.0618
Epoch: 39, Loss: 5.7998
Epoch: 40, Loss: 5.8250
Epoch: 41, Loss:

In [39]:
def prediction(model, vocab, text):
    # Set model to evaluation mode
    model.eval()

    # Tokenize
    tokenized_text = word_tokenize(text.lower())

    # Text -> Numerical indices
    numerical_text = text_to_indices(tokenized_text, vocab)

    # Padding
    padded_text = torch.tensor([0] * (61 - len(numerical_text)) + numerical_text, dtype=torch.long).unsqueeze(0)

    # Move tensor to the same device as model
    device = next(model.parameters()).device  # Get model's device
    padded_text = padded_text.to(device)

    # Disable gradient calculation for inference
    with torch.no_grad():
        # Send to model
        output = model(padded_text)

        # Predicted index
        value, index = torch.max(output, dim=1)

    # Move index back to CPU for vocabulary lookup if needed
    index = index.item()  # Get the scalar value

    # Merge with text
    return text + " " + list(vocab.keys())[index]

In [40]:
prediction(model, vocab, "The course follows a monthly")

'The course follows a monthly subscription'

In [41]:
import time

num_tokens = 10
input_text = "hi how are"

for i in range(num_tokens):
  output_text = prediction(model, vocab, input_text)
  print(output_text)
  input_text = output_text
  time.sleep(0.5)


hi how are 2
hi how are 2 criterias
hi how are 2 criterias :
hi how are 2 criterias : :
hi how are 2 criterias : : ,
hi how are 2 criterias : : , can
hi how are 2 criterias : : , can i
hi how are 2 criterias : : , can i pay
hi how are 2 criterias : : , can i pay the
hi how are 2 criterias : : , can i pay the payment


In [42]:
dataloader1 = DataLoader(dataset, batch_size=32, shuffle=False)

In [43]:
# Function to calculate accuracy
def calculate_accuracy(model, dataloader, device):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # No need to compute gradients
        for batch_x, batch_y in dataloader1:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)

            # Get model predictions
            outputs = model(batch_x)

            # Get the predicted word indices
            _, predicted = torch.max(outputs, dim=1)

            # Compare with actual labels
            correct += (predicted == batch_y).sum().item()
            total += batch_y.size(0)

    accuracy = correct / total * 100
    return accuracy

# Compute accuracy
accuracy = calculate_accuracy(model, dataloader, device)
print(f"Model Accuracy: {accuracy:.2f}%")


Model Accuracy: 95.65%
